# Import library and split data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/BPD.csv')
df

,Send Rates,Block Size,Avg Latency,Throughput
0,10.0,10,0.32,10.0
1,10.0,20,0.82,10.0
2,10.0,30,1.18,9.8
3,10.0,40,1.18,9.8
4,10.0,50,1.18,9.8
...,...,...,...,...
3115,194.1,760,2.57,153.0
3116,200.7,770,2.33,154.8
3117,195.1,780,2.17,158.4
3118,192.4,790,2.26,156.3


In [5]:
xy = df.values
xy

array([[1.000e+01, 1.000e+01, 3.200e-01, 1.000e+01],
       [1.000e+01, 2.000e+01, 8.200e-01, 1.000e+01],
       [1.000e+01, 3.000e+01, 1.180e+00, 9.800e+00],
       ...,
       [1.951e+02, 7.800e+02, 2.170e+00, 1.584e+02],
       [1.924e+02, 7.900e+02, 2.260e+00, 1.563e+02],
       [1.925e+02, 8.000e+02, 2.020e+00, 1.571e+02]])

In [6]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [7]:
# set random seed
seed = 42
torch.manual_seed(seed)

In [8]:
# Read raw data and divide training set and test set
raw_data = xy
X = raw_data[:, :2]
Y1 = raw_data[:, -2:-1]
Y2 = raw_data[:, -1:]

In [9]:
X

array([[ 10. ,  10. ],
       [ 10. ,  20. ],
       [ 10. ,  30. ],
       ...,
       [195.1, 780. ],
       [192.4, 790. ],
       [192.5, 800. ]])

In [10]:
Y1

array([[0.32],
       [0.82],
       [1.18],
       ...,
       [2.17],
       [2.26],
       [2.02]])

In [11]:
Y2

array([[ 10. ],
       [ 10. ],
       [  9.8],
       ...,
       [158.4],
       [156.3],
       [157.1]])

# MLP1(send rates, block size) = latency

In [12]:
# Split training set and test set
Xtrain1, Xtest1, Ytrain1, Ytest1 = train_test_split(X, Y1, test_size=0.2, random_state=1)

In [13]:
# test set
Xtest1

array([[190.2, 720. ],
       [ 15.1, 310. ],
       [ 75.3,  40. ],
       ...,
       [105.4, 440. ],
       [100.1,  80. ],
       [110.5,  20. ]])

In [14]:
Ytest1

array([[2.99],
       [1.18],
       [0.33],
       [1.16],
       [1.15],
       [1.55],
       [0.9 ],
       [0.81],
       [1.2 ],
       [1.22],
       [1.23],
       [1.28],
       [0.92],
       [1.18],
       [0.91],
       [0.26],
       [1.18],
       [1.26],
       [1.17],
       [0.87],
       [1.18],
       [0.92],
       [0.94],
       [1.24],
       [0.88],
       [1.19],
       [2.32],
       [1.01],
       [1.66],
       [0.76],
       [0.77],
       [1.34],
       [0.89],
       [1.19],
       [0.65],
       [0.83],
       [1.13],
       [2.57],
       [0.92],
       [1.2 ],
       [0.97],
       [0.89],
       [0.88],
       [1.16],
       [2.29],
       [0.47],
       [1.15],
       [0.93],
       [0.85],
       [0.8 ],
       [1.19],
       [2.56],
       [1.18],
       [1.73],
       [1.23],
       [1.45],
       [2.1 ],
       [1.27],
       [1.52],
       [0.85],
       [1.17],
       [0.95],
       [0.86],
       [0.25],
       [0.84],
       [0.9 ],
       [1.

In [15]:
# property scaling
min_max_scaler1 = preprocessing.MinMaxScaler()

In [16]:
# Scaling training set data
Xtrain1_minmax = min_max_scaler1.fit_transform(Xtrain1)
Xtrain1_minmax

array([[0.8427673 , 0.51898734],
       [0.39412998, 0.92405063],
       [0.52568134, 0.46835443],
       ...,
       [0.34224319, 0.70886076],
       [0.05293501, 0.94936709],
       [0.34224319, 0.26582278]])

In [17]:
# Apply the same scaling to the test set data
Xtest1_minmax = min_max_scaler1.transform(Xtest1)
Xtest1_minmax

array([[0.94444444, 0.89873418],
       [0.02672956, 0.37974684],
       [0.34224319, 0.03797468],
       ...,
       [0.5       , 0.5443038 ],
       [0.47222222, 0.08860759],
       [0.52672956, 0.01265823]])

In [18]:
Xtest1_tensor = torch.from_numpy(Xtest1_minmax).type(torch.float32)
Ytest1_tensor = torch.from_numpy(Ytest1).type(torch.float32)

In [19]:
Xtest1_tensor

tensor([[0.9444, 0.8987],
        [0.0267, 0.3797],
        [0.3422, 0.0380],
        ...,
        [0.5000, 0.5443],
        [0.4722, 0.0886],
        [0.5267, 0.0127]])

In [20]:
# batch the training dataset
# prepare dataset
class BlockChainDataset(Dataset):
    def __init__(self, data, label):
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(data).type(torch.float32)
        self.y_data = torch.from_numpy(label).type(torch.float32)
        
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [21]:
train_dataset = BlockChainDataset(Xtrain1_minmax, Ytrain1)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

In [22]:
import torch.nn as nn
import torch.optim as optim

In [23]:
# design model using class
class Lat(torch.nn.Module):
    def __init__(self):
        super(Lat, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 8)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
    
    
model = Lat()

In [69]:
# net = nn.Sequential(
#     nn.Linear(2, 64), nn.BatchNorm1d(64), nn.Dropout(p=0.5), nn.ReLU(),
#     nn.Linear(64, 64), nn.BatchNorm1d(64), nn.Dropout(p=0.5), nn.ReLU(),
#     nn.Linear(64, 1))


# model = net

In [24]:
# construct loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
# traning cycle forward, backward, update
def train(epoch):
    train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader, 0):
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.shape[0]
    if (epoch + 1) % 10 == 0:
        print('epoch:', epoch + 1, 'train_loss:', train_loss / len(Xtrain1))
        

def test():
    with torch.no_grad():
        y_pred = model(Xtest1_tensor)
        loss = criterion(y_pred, Ytest1_tensor)
        print('test_loss:', loss)

In [30]:
if __name__ == '__main__':
    for epoch in range(1000):
        train(epoch)
        if (epoch + 1) % 10 == 0:
            model.eval()
            test()

epoch: 10 train_loss: 0.047569985513408214
test_loss: tensor(0.0288)
epoch: 20 train_loss: 0.04749157062421242
test_loss: tensor(0.0287)
epoch: 30 train_loss: 0.04744447714004379
test_loss: tensor(0.0287)
epoch: 40 train_loss: 0.047379278404733695
test_loss: tensor(0.0286)
epoch: 50 train_loss: 0.04730947837472344
test_loss: tensor(0.0286)
epoch: 60 train_loss: 0.04723189055370406
test_loss: tensor(0.0286)
epoch: 70 train_loss: 0.047188885969659075
test_loss: tensor(0.0285)
epoch: 80 train_loss: 0.04720258188004104
test_loss: tensor(0.0284)
epoch: 90 train_loss: 0.047091191909156546
test_loss: tensor(0.0285)
epoch: 100 train_loss: 0.04707555804186715
test_loss: tensor(0.0284)
epoch: 110 train_loss: 0.04701771322661676
test_loss: tensor(0.0284)
epoch: 120 train_loss: 0.046914102658868216
test_loss: tensor(0.0284)
epoch: 130 train_loss: 0.04687392701490376
test_loss: tensor(0.0284)
epoch: 140 train_loss: 0.04681459322380714
test_loss: tensor(0.0283)
epoch: 150 train_loss: 0.0467265389668

In [27]:
Xtest1

array([[190.2, 720. ],
       [ 15.1, 310. ],
       [ 75.3,  40. ],
       ...,
       [105.4, 440. ],
       [100.1,  80. ],
       [110.5,  20. ]])

In [28]:
Xtest1[:, 1].reshape(-1)

array([720., 310.,  40., 370., 210., 470., 350., 380., 390., 460., 610.,
       480., 440., 140., 470.,  30., 740., 580., 470., 100., 460., 630.,
       260., 560., 480.,  50., 450., 790., 280.,  80., 310., 490., 260.,
        70.,  90., 370., 690., 760., 700., 760., 230., 600., 560., 390.,
       180.,  50., 770., 590., 350., 580., 200., 670.,  40.,  90., 200.,
       110., 200., 560., 570., 370., 110., 190., 220.,  20., 290., 160.,
       340., 560., 550., 130., 540., 390., 730.,  10., 780., 140., 770.,
       430., 170., 680.,  90., 140., 700., 370., 190., 220., 370.,  50.,
       230., 410., 730., 180., 170., 290., 190., 570., 120., 360.,  20.,
       690., 410., 290., 780., 450., 160., 320., 790., 470., 540., 320.,
       740., 380.,  50., 510.,  70., 290., 110.,  60., 700., 660., 120.,
       580., 120., 710., 160., 670.,  90., 490., 570., 540., 670., 230.,
        80., 420., 180., 450., 650., 550., 440.,  20., 640.,  10., 220.,
       320., 440., 520., 740., 240., 540., 480., 19

In [31]:
with torch.no_grad():
    y_pred = model(Xtest1_tensor)
y = pd.concat([pd.Series(Xtest1[:, 0].reshape(-1), name='send rates'), pd.Series(Xtest1[:, 1].reshape(-1), name='block size'), 
               pd.Series(Ytest1_tensor.numpy().reshape(-1), name='latency_true'), pd.Series(y_pred.numpy().reshape(-1), name='latency_pred')], 
              axis=1)
y.to_csv('./latency_true_pred.csv')

In [2]:
# Compute test set MAE RMSE MAPE
import pandas as pd

data = pd.read_csv('./latency_true_pred.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,send rates,block size,latency_true,latency_pred
0,190.2,720.0,2.99,2.400978
1,15.1,310.0,1.18,1.151520
2,75.3,40.0,0.33,0.334008
3,40.2,370.0,1.16,1.160947
4,30.1,210.0,1.15,1.150524


In [3]:
y = data['latency_true'].values
y

array([2.99, 1.18, 0.33, 1.16, 1.15, 1.55, 0.9 , 0.81, 1.2 , 1.22, 1.23,
       1.28, 0.92, 1.18, 0.91, 0.26, 1.18, 1.26, 1.17, 0.87, 1.18, 0.92,
       0.94, 1.24, 0.88, 1.19, 2.32, 1.01, 1.66, 0.76, 0.77, 1.34, 0.89,
       1.19, 0.65, 0.83, 1.13, 2.57, 0.92, 1.2 , 0.97, 0.89, 0.88, 1.16,
       2.29, 0.47, 1.15, 0.93, 0.85, 0.8 , 1.19, 2.56, 1.18, 1.73, 1.23,
       1.45, 2.1 , 1.27, 1.52, 0.85, 1.17, 0.95, 0.86, 0.25, 0.84, 0.9 ,
       1.09, 0.84, 1.24, 0.91, 1.2 , 1.25, 1.58, 2.9 , 1.59, 1.1 , 2.33,
       1.21, 1.25, 0.87, 2.18, 1.25, 0.86, 1.38, 2.14, 1.48, 1.26, 1.18,
       0.93, 1.  , 1.11, 1.21, 1.03, 0.81, 1.17, 1.24, 1.17, 1.24, 0.45,
       1.31, 1.15, 1.3 , 2.17, 0.9 , 1.07, 2.36, 1.17, 1.77, 1.25, 1.68,
       1.89, 0.89, 0.43, 0.92, 0.59, 1.16, 1.6 , 0.67, 0.94, 0.96, 1.04,
       0.95, 0.97, 0.87, 0.88, 1.16, 0.8 , 0.92, 1.17, 0.84, 1.16, 0.86,
       1.66, 0.95, 1.88, 0.87, 1.12, 1.17, 1.18, 1.87, 1.75, 3.07, 0.94,
       0.93, 0.87, 1.27, 0.8 , 1.24, 1.18, 1.02, 0.

In [4]:
y_hat = data['latency_pred'].values
y_hat

array([2.400978  , 1.1515205 , 0.33400798, 1.1609472 , 1.1505245 ,
       1.7063725 , 0.8917548 , 0.93338865, 1.1816051 , 1.1858441 ,
       1.216482  , 1.2183396 , 0.92686284, 1.1954347 , 0.9042676 ,
       0.21569604, 1.1397014 , 1.2525133 , 1.1491901 , 0.6666734 ,
       1.1505215 , 0.91166484, 0.941359  , 1.4415953 , 0.9020423 ,
       1.7602578 , 2.0381522 , 1.2129923 , 2.0060003 , 0.9479116 ,
       1.0600975 , 1.3618059 , 0.92794514, 1.185236  , 0.6052137 ,
       0.9404031 , 1.1657209 , 2.3476548 , 0.9376341 , 1.1683358 ,
       1.0390375 , 1.0686388 , 0.9108851 , 1.1456023 , 2.2874742 ,
       0.6233517 , 1.1445891 , 0.9518882 , 0.9364728 , 0.8699529 ,
       1.1538765 , 2.2778468 , 1.0669134 , 2.0118184 , 1.180171  ,
       1.3776397 , 2.1200986 , 1.2513047 , 1.7362957 , 0.9224442 ,
       1.1466931 , 0.9323207 , 0.9109255 , 0.18951967, 0.9981762 ,
       0.91919714, 1.0787201 , 1.0071461 , 1.230914  , 0.85905963,
       1.1916963 , 1.2010525 , 1.5030155 , 1.8865573 , 2.03878

In [5]:
from sklearn import metrics

In [7]:
MAE = metrics.mean_absolute_error(y, y_hat)
RMSE = metrics.mean_squared_error(y, y_hat) ** 0.5
MAPE = metrics.mean_absolute_percentage_error(y, y_hat)
print('MAE: ', MAE)
print('RMSE: ', RMSE)
print('MAPE: ', MAPE)

MAE:  0.09403885004807692
RMSE:  0.16596019501329495
MAPE:  0.07551825326603377


# MLP2(send rates, block size) = throughput

In [32]:
# Split training set and test set
Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X, Y2, test_size=0.2, random_state=1)

In [33]:
Xtest2

array([[190.2, 720. ],
       [ 15.1, 310. ],
       [ 75.3,  40. ],
       ...,
       [105.4, 440. ],
       [100.1,  80. ],
       [110.5,  20. ]])

In [34]:
Ytest2

array([[138. ],
       [ 14.8],
       [ 74.9],
       [ 37.1],
       [ 28.7],
       [155.6],
       [ 97.4],
       [ 98.4],
       [ 43.4],
       [ 43.3],
       [ 48.4],
       [120.4],
       [ 69.2],
       [  9.8],
       [ 95.8],
       [ 83.4],
       [ 28.3],
       [ 59.4],
       [ 14.7],
       [109.6],
       [ 10. ],
       [ 96.3],
       [ 79.2],
       [119.5],
       [ 81.7],
       [123.9],
       [148.3],
       [119. ],
       [155.9],
       [113. ],
       [122.2],
       [119.7],
       [ 80.8],
       [  9.8],
       [ 97. ],
       [ 80.9],
       [ 69.4],
       [153. ],
       [ 79.9],
       [ 10. ],
       [118.9],
       [121.8],
       [ 81.8],
       [ 34.9],
       [152.9],
       [109.1],
       [ 28.5],
       [ 69.1],
       [ 81.3],
       [ 97.4],
       [ 14.7],
       [142.7],
       [  9.8],
       [123.5],
       [ 60.6],
       [115. ],
       [148.8],
       [ 53.6],
       [153.4],
       [ 97.5],
       [ 49.8],
       [ 79.6],
       [

In [35]:
# property scaling
min_max_scaler2 = preprocessing.MinMaxScaler()

In [36]:
# Scaling training set data
Xtrain2_minmax = min_max_scaler2.fit_transform(Xtrain2)
Xtrain2_minmax

array([[0.8427673 , 0.51898734],
       [0.39412998, 0.92405063],
       [0.52568134, 0.46835443],
       ...,
       [0.34224319, 0.70886076],
       [0.05293501, 0.94936709],
       [0.34224319, 0.26582278]])

In [37]:
# Apply the same scaling to the test set data
Xtest2_minmax = min_max_scaler2.transform(Xtest2)
Xtest2_minmax

array([[0.94444444, 0.89873418],
       [0.02672956, 0.37974684],
       [0.34224319, 0.03797468],
       ...,
       [0.5       , 0.5443038 ],
       [0.47222222, 0.08860759],
       [0.52672956, 0.01265823]])

In [38]:
Xtest2_tensor = torch.from_numpy(Xtest2_minmax).type(torch.float32)
Ytest2_tensor = torch.from_numpy(Ytest2).type(torch.float32)

In [39]:
train_dataset = BlockChainDataset(Xtrain2_minmax, Ytrain2)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

In [40]:
# design model using class
class Thr(torch.nn.Module):
    def __init__(self):
        super(Thr, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 8)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
    
    
model1 = Thr()

In [41]:
# construct loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [42]:
# traning cycle forward, backward, update
def train1(epoch):
    train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader, 0):
        y_pred = model1(inputs)
        loss = criterion(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.shape[0]
    if (epoch + 1) % 10 == 0:
        print('epoch:', epoch + 1, 'train_loss:', train_loss / len(Xtrain2))
        

def test1():
    with torch.no_grad():
        y_pred = model1(Xtest2_tensor)
        loss = criterion(y_pred, Ytest2_tensor)
        print('test_loss:', loss)

In [43]:
if __name__ == '__main__':
    for epoch in range(1000):
        train1(epoch)
        if (epoch + 1) % 10 == 0:
            model1.eval()
            test1()

epoch: 10 train_loss: 2624.165780874399
test_loss: tensor(1049.7826)
epoch: 20 train_loss: 118.51343037531926
test_loss: tensor(106.5045)
epoch: 30 train_loss: 55.39982763926188
test_loss: tensor(50.2529)
epoch: 40 train_loss: 51.34020211146428
test_loss: tensor(46.4299)
epoch: 50 train_loss: 50.01990961417174
test_loss: tensor(45.1452)
epoch: 60 train_loss: 49.140734403561325
test_loss: tensor(44.3736)
epoch: 70 train_loss: 48.43642929272774
test_loss: tensor(43.7726)
epoch: 80 train_loss: 47.85381524990766
test_loss: tensor(43.2812)
epoch: 90 train_loss: 47.393269636692146
test_loss: tensor(42.8932)
epoch: 100 train_loss: 47.01151505494729
test_loss: tensor(42.6121)
epoch: 110 train_loss: 46.68488443814791
test_loss: tensor(42.4329)
epoch: 120 train_loss: 46.39409241309533
test_loss: tensor(42.3292)
epoch: 130 train_loss: 46.14913514944223
test_loss: tensor(42.2336)
epoch: 140 train_loss: 45.954646232800606
test_loss: tensor(42.1533)
epoch: 150 train_loss: 45.7958901478694
test_loss:

In [44]:
with torch.no_grad():
    y_pred = model1(Xtest2_tensor)
y = pd.concat([pd.Series(Xtest2[:, 0].reshape(-1), name='send rates'), pd.Series(Xtest2[:, 1].reshape(-1), name='block size'), 
               pd.Series(Ytest2_tensor.numpy().reshape(-1), name='throughput_true'), 
               pd.Series(y_pred.numpy().reshape(-1), name='throughput_pred')], axis=1)
y.to_csv('./throughput_true_pred.csv')

In [8]:
# Compute the test set MAE RMSE MAPE
import pandas as pd

data = pd.read_csv('./throughput_true_pred.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,send rates,block size,throughput_true,throughput_pred
0,190.2,720.0,138.0,152.887860
1,15.1,310.0,14.8,15.589494
2,75.3,40.0,74.9,68.419890
3,40.2,370.0,37.1,37.523610
4,30.1,210.0,28.7,28.770885


In [9]:
y = data['throughput_true'].values
y

array([138. ,  14.8,  74.9,  37.1,  28.7, 155.6,  97.4,  98.4,  43.4,
        43.3,  48.4, 120.4,  69.2,   9.8,  95.8,  83.4,  28.3,  59.4,
        14.7, 109.6,  10. ,  96.3,  79.2, 119.5,  81.7, 123.9, 148.3,
       119. , 155.9, 113. , 122.2, 119.7,  80.8,   9.8,  97. ,  80.9,
        69.4, 153. ,  79.9,  10. , 118.9, 121.8,  81.8,  34.9, 152.9,
       109.1,  28.5,  69.1,  81.3,  97.4,  14.7, 142.7,   9.8, 123.5,
        60.6, 115. , 148.8,  53.6, 153.4,  97.5,  49.8,  79.6,  81.2,
        40.1, 122.1, 120. ,  61.3, 120.6,  53.3,  80.4,  43.4,  48.4,
       118.9, 124. , 156.4,  61.7, 154.8,  43.6,  60.8,  97. , 128.5,
        61.3,  96.7, 120.1, 159.1, 153.3,  53.3,   9.8,  69.4,  69. ,
        61.3,  43.6,  97. ,  97.3,  14.8,  60.9,  19.9,  47.5,  20.1,
       118.6,  34.7, 119.9, 158.4, 119.1, 119.5, 148.8,  37.1, 118. ,
        52.6, 151.9, 155.7,  82.1,  92.9,  81.6,  69.5,   9.9, 118.9,
        51. , 119.1, 122.3,  78.2,  96.7,  91.7, 120.2,  96.9,  24.3,
        61.6,  95.3,

In [10]:
y_hat = data['throughput_pred'].values
y_hat

array([152.88786  ,  15.589494 ,  68.41989  ,  37.52361  ,  28.770885 ,
       139.68205  , 104.47056  ,  90.202835 ,  41.88862  ,  41.850822 ,
        46.14564  , 129.10866  ,  72.579735 ,  11.217959 ,  97.329796 ,
        85.52037  ,  28.484726 ,  54.913433 ,  15.503114 , 107.539215 ,
        11.04519  , 100.51912  ,  77.05273  , 132.39178  ,  77.021454 ,
       121.76822  , 144.64777  , 124.74392  , 143.58952  , 113.23081  ,
       122.070435 , 132.67612  ,  81.42852  ,  11.255758 , 101.591095 ,
        85.83244  ,  63.693157 , 155.54956  ,  81.27847  ,  10.883223 ,
       122.44563  , 118.098495 ,  76.890755 ,  33.137016 , 141.28368  ,
       105.57944  ,  28.468533 ,  72.498764 ,  85.84325  ,  89.99848  ,
        15.648882 , 142.01544  ,  11.271953 , 128.22267  ,  59.494392 ,
       120.93217  , 140.23885  ,  55.011745 , 139.19548  ,  93.94701  ,
        50.79139  ,  77.178024 ,  85.91342  ,  37.71258  , 118.80491  ,
       115.29721  ,  63.882126 , 118.039856 ,  50.553825 ,  81.4

In [11]:
from sklearn import metrics

In [12]:
MAE = metrics.mean_absolute_error(y, y_hat)
RMSE = metrics.mean_squared_error(y, y_hat) ** 0.5
MAPE = metrics.mean_absolute_percentage_error(y, y_hat)
print('MAE: ', MAE)
print('RMSE: ', RMSE)
print('MAPE: ', MAPE)

MAE:  4.316324364583333
RMSE:  6.037723286365757
MAPE:  0.04614929654089586


# Choose the optimal block size

## MLP1 to predict latency

In [250]:
rng = pd.date_range('2014/09/25 00:05',periods=1728,freq ='5T')
rng

DatetimeIndex(['2014-09-25 00:05:00', '2014-09-25 00:10:00',
               '2014-09-25 00:15:00', '2014-09-25 00:20:00',
               '2014-09-25 00:25:00', '2014-09-25 00:30:00',
               '2014-09-25 00:35:00', '2014-09-25 00:40:00',
               '2014-09-25 00:45:00', '2014-09-25 00:50:00',
               ...
               '2014-09-30 23:15:00', '2014-09-30 23:20:00',
               '2014-09-30 23:25:00', '2014-09-30 23:30:00',
               '2014-09-30 23:35:00', '2014-09-30 23:40:00',
               '2014-09-30 23:45:00', '2014-09-30 23:50:00',
               '2014-09-30 23:55:00', '2014-10-01 00:00:00'],
              dtype='datetime64[ns]', length=1728, freq='5T')

In [271]:
send_rates_true_pred = pd.read_csv("../result/y_true_pred.csv")
send_rates_true_pred['datetime'] = rng
send_rates_true_pred.head()

,Unnamed: 0,y_true,y_pred,datetime
0,0,17937.000832,19101.491395,2014-09-25 00:05:00
1,1,19263.999929,18565.845787,2014-09-25 00:10:00
2,2,16234.000265,19006.439372,2014-09-25 00:15:00
3,3,14814.000125,17462.212608,2014-09-25 00:20:00
4,4,13504.000112,16249.269695,2014-09-25 00:25:00


In [272]:
len(send_rates_true_pred)

1728

In [273]:
send_rates_pred = send_rates_true_pred['y_pred'].values
send_rates_pred = send_rates_pred / 300
send_rates_pred = list(send_rates_pred)
send_rates_pred

[63.6716379832228,
 61.88615262200435,
 63.35479790707429,
 58.20737536062797,
 54.16423231770595,
 50.174860953440266,
 45.27682748039563,
 51.27762869273623,
 50.86778451402982,
 45.99366690625747,
 40.737415696879225,
 42.401347869535286,
 39.51747398888072,
 38.65573471595844,
 37.21748384853204,
 35.19733534400661,
 32.82875272711118,
 31.145713874374827,
 27.91393989523252,
 25.862601307680208,
 24.689389639695484,
 24.468197723378736,
 23.694803320144615,
 21.49829864365359,
 22.3021547900637,
 20.573120357667406,
 20.91246705328425,
 19.355611549591025,
 18.856039267530043,
 17.076497679737706,
 17.009775781904658,
 16.377620399904746,
 18.013960580204923,
 16.73137552452584,
 13.510642932703096,
 14.958984979366262,
 15.144111066994567,
 18.03539152128001,
 18.37162441102167,
 15.89068619137009,
 16.443546632354458,
 17.387844604489704,
 16.252848591854175,
 16.102981191587943,
 14.88998242912193,
 14.813609263201554,
 15.35743125145634,
 16.142980973546702,
 16.43811162573596

In [274]:
send_rates_pred_all = []
for i in range(len(send_rates_pred)):
    send_rates_pred_all.extend([send_rates_pred[i]] * 80)

In [275]:
send_rates_pred_all

[63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.6716379832228,
 63.67163798

In [276]:
len(send_rates_pred_all)

138240

In [277]:
BS = list(range(10, 810, 10)) * 1728
BS

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500,
 510,
 520,
 530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500,
 510,
 520,
 530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 

In [278]:
len(BS)

138240

In [280]:
datetime = send_rates_true_pred['datetime'].values
datetime = list(datetime)
datetime

[numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:10:00.000000000'),
 numpy.datetime64('2014-09-25T00:15:00.000000000'),
 numpy.datetime64('2014-09-25T00:20:00.000000000'),
 numpy.datetime64('2014-09-25T00:25:00.000000000'),
 numpy.datetime64('2014-09-25T00:30:00.000000000'),
 numpy.datetime64('2014-09-25T00:35:00.000000000'),
 numpy.datetime64('2014-09-25T00:40:00.000000000'),
 numpy.datetime64('2014-09-25T00:45:00.000000000'),
 numpy.datetime64('2014-09-25T00:50:00.000000000'),
 numpy.datetime64('2014-09-25T00:55:00.000000000'),
 numpy.datetime64('2014-09-25T01:00:00.000000000'),
 numpy.datetime64('2014-09-25T01:05:00.000000000'),
 numpy.datetime64('2014-09-25T01:10:00.000000000'),
 numpy.datetime64('2014-09-25T01:15:00.000000000'),
 numpy.datetime64('2014-09-25T01:20:00.000000000'),
 numpy.datetime64('2014-09-25T01:25:00.000000000'),
 numpy.datetime64('2014-09-25T01:30:00.000000000'),
 numpy.datetime64('2014-09-25T01:35:00.000000000'),
 numpy.datet

In [281]:
datetime_all = []
for i in range(len(datetime)):
    datetime_all.extend([datetime[i]] * 80)

In [282]:
datetime_all

[numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datetime64('2014-09-25T00:05:00.000000000'),
 numpy.datet

In [298]:
XpBS = {"Xp": send_rates_pred_all, "BS": BS, "datetime": datetime_all}
XpBS

{'Xp': [63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.6716379832228,
  63.671637983

In [299]:
XpBS = pd.DataFrame(XpBS)
XpBS

,Xp,BS,datetime
0,63.671638,10,2014-09-25 00:05:00
1,63.671638,20,2014-09-25 00:05:00
2,63.671638,30,2014-09-25 00:05:00
3,63.671638,40,2014-09-25 00:05:00
4,63.671638,50,2014-09-25 00:05:00
...,...,...,...
138235,49.815040,760,2014-10-01 00:00:00
138236,49.815040,770,2014-10-01 00:00:00
138237,49.815040,780,2014-10-01 00:00:00
138238,49.815040,790,2014-10-01 00:00:00


In [300]:
XpBS.set_index('datetime', drop=True, inplace=True)
XpBS

,Xp,BS
datetime,,
2014-09-25 00:05:00,63.671638,10
2014-09-25 00:05:00,63.671638,20
2014-09-25 00:05:00,63.671638,30
2014-09-25 00:05:00,63.671638,40
2014-09-25 00:05:00,63.671638,50
...,...,...
2014-10-01 00:00:00,49.815040,760
2014-10-01 00:00:00,49.815040,770
2014-10-01 00:00:00,49.815040,780


In [301]:
XpBS_values = XpBS.values
XpBS_values

array([[ 63.67163798,  10.        ],
       [ 63.67163798,  20.        ],
       [ 63.67163798,  30.        ],
       ...,
       [ 49.81503965, 780.        ],
       [ 49.81503965, 790.        ],
       [ 49.81503965, 800.        ]])

In [302]:
# Normalization
XpBS_minmax = min_max_scaler1.transform(XpBS_values)
XpBS_minmax

array([[0.28129789, 0.        ],
       [0.28129789, 0.01265823],
       [0.28129789, 0.02531646],
       ...,
       [0.20867421, 0.97468354],
       [0.20867421, 0.98734177],
       [0.20867421, 1.        ]])

In [303]:
XpBS_tensor = torch.from_numpy(XpBS_minmax).type(torch.float32)

In [304]:
with torch.no_grad():
    latency_pred = model(XpBS_tensor)
y = pd.concat([pd.Series(XpBS_values[:, 0].reshape(-1), name='send rates'), pd.Series(XpBS_values[:, 1].reshape(-1), name='block size'), 
               pd.Series(latency_pred.numpy().reshape(-1), name='latency_pred')], axis=1)
y.index = XpBS.index
y.to_csv('./latency_pred.csv')

In [305]:
y

,send rates,block size,latency_pred
datetime,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678
2014-09-25 00:05:00,63.671638,20.0,0.175356
2014-09-25 00:05:00,63.671638,30.0,0.307779
2014-09-25 00:05:00,63.671638,40.0,0.392023
2014-09-25 00:05:00,63.671638,50.0,0.487231
...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382
2014-10-01 00:00:00,49.815040,770.0,1.209036
2014-10-01 00:00:00,49.815040,780.0,1.207692


## MLP2 to predict throughput

In [306]:
# Normalization
XpBS_minmax2 = min_max_scaler2.transform(XpBS_values)
XpBS_minmax2

array([[0.28129789, 0.        ],
       [0.28129789, 0.01265823],
       [0.28129789, 0.02531646],
       ...,
       [0.20867421, 0.97468354],
       [0.20867421, 0.98734177],
       [0.20867421, 1.        ]])

In [307]:
XpBS_tensor2 = torch.from_numpy(XpBS_minmax2).type(torch.float32)

In [308]:
with torch.no_grad():
    throughput_pred = model1(XpBS_tensor2)
y = pd.concat([pd.Series(XpBS_values[:, 0].reshape(-1), name='send rates'), pd.Series(XpBS_values[:, 1].reshape(-1), name='block size'), 
               pd.Series(throughput_pred.numpy().reshape(-1), name='throughput_pred')], axis=1)
y.index = XpBS.index
y.to_csv('./throughput_pred.csv')

In [309]:
y

,send rates,block size,throughput_pred
datetime,,,
2014-09-25 00:05:00,63.671638,10.0,58.259411
2014-09-25 00:05:00,63.671638,20.0,58.254017
2014-09-25 00:05:00,63.671638,30.0,58.248619
2014-09-25 00:05:00,63.671638,40.0,58.243221
2014-09-25 00:05:00,63.671638,50.0,58.237820
...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,45.727749
2014-10-01 00:00:00,49.815040,770.0,45.722359
2014-10-01 00:00:00,49.815040,780.0,45.716946


In [310]:
with torch.no_grad():
    latency_pred = model(XpBS_tensor)
    throughput_pred = model1(XpBS_tensor2)
y = pd.concat([pd.Series(XpBS_values[:, 0].reshape(-1), name='send rates'), pd.Series(XpBS_values[:, 1].reshape(-1), name='block size'), 
               pd.Series(latency_pred.numpy().reshape(-1), name='latency_pred'),
               pd.Series(throughput_pred.numpy().reshape(-1), name='throughput_pred')], axis=1)
y.index = XpBS.index
y.to_csv('./latency_throughput_pred.csv')

## Set the score function to chose the best block

In [311]:
y

,send rates,block size,latency_pred,throughput_pred
datetime,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259411
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248619
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243221
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820
...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727749
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722359
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946


In [312]:
for name, group in y.groupby('send rates', sort=False):
    print(name)
    print(group)

63.6716379832228
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-25 00:05:00   63.671638        10.0      0.118678        58.259411
2014-09-25 00:05:00   63.671638        20.0      0.175356        58.254017
2014-09-25 00:05:00   63.671638        30.0      0.307779        58.248619
2014-09-25 00:05:00   63.671638        40.0      0.392023        58.243221
2014-09-25 00:05:00   63.671638        50.0      0.487231        58.237820
...                         ...         ...           ...              ...
2014-09-25 00:05:00   63.671638       760.0      1.260267        57.854485
2014-09-25 00:05:00   63.671638       770.0      1.260168        57.849087
2014-09-25 00:05:00   63.671638       780.0      1.259605        57.843689
2014-09-25 00:05:00   63.671638       790.0      1.258261        57.838299
2014-09-25 00:05:00   63.671638       800.0      1.256918        57.832890

[80 row

[80 rows x 4 columns]
13.510642932703096
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-25 02:55:00   13.510643        10.0      0.314256        14.398872
2014-09-25 02:55:00   13.510643        20.0      0.573254        14.355126
2014-09-25 02:55:00   13.510643        30.0      0.816542        14.349727
2014-09-25 02:55:00   13.510643        40.0      1.048308        14.344327
2014-09-25 02:55:00   13.510643        50.0      1.125414        14.338927
...                         ...         ...           ...              ...
2014-09-25 02:55:00   13.510643       760.0      1.162762        13.955601
2014-09-25 02:55:00   13.510643       770.0      1.163571        13.950191
2014-09-25 02:55:00   13.510643       780.0      1.164384        13.944801
2014-09-25 02:55:00   13.510643       790.0      1.165197        13.939396
2014-09-25 02:55:00   13.510643       800.0      1.166009  

[80 rows x 4 columns]
48.64889715825518
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-25 06:15:00   48.648897        10.0      0.118678        45.112118
2014-09-25 06:15:00   48.648897        20.0      0.177180        45.106716
2014-09-25 06:15:00   48.648897        30.0      0.327881        45.101318
2014-09-25 06:15:00   48.648897        40.0      0.437549        45.095913
2014-09-25 06:15:00   48.648897        50.0      0.522931        45.090519
...                         ...         ...           ...              ...
2014-09-25 06:15:00   48.648897       760.0      1.206013        44.707195
2014-09-25 06:15:00   48.648897       770.0      1.204666        44.701786
2014-09-25 06:15:00   48.648897       780.0      1.203323        44.696384
2014-09-25 06:15:00   48.648897       790.0      1.201979        44.690987
2014-09-25 06:15:00   48.648897       800.0      1.200635   

[80 rows x 4 columns]
118.8951550235351
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-25 08:50:00  118.895155        10.0      0.187202        93.626373
2014-09-25 08:50:00  118.895155        20.0      0.300922        94.527969
2014-09-25 08:50:00  118.895155        30.0      0.356238        95.429565
2014-09-25 08:50:00  118.895155        40.0      0.414900        96.331177
2014-09-25 08:50:00  118.895155        50.0      0.439323        97.232773
...                         ...         ...           ...              ...
2014-09-25 08:50:00  118.895155       760.0      0.943275       102.640984
2014-09-25 08:50:00  118.895155       770.0      0.949017       102.620476
2014-09-25 08:50:00  118.895155       780.0      0.954771       102.599960
2014-09-25 08:50:00  118.895155       790.0      0.960525       102.579437
2014-09-25 08:50:00  118.895155       800.0      0.966279   

[80 rows x 4 columns]
90.53057770093282
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-25 12:15:00   90.530578        10.0      0.118678        76.994568
2014-09-25 12:15:00   90.530578        20.0      0.162967        77.911285
2014-09-25 12:15:00   90.530578        30.0      0.221054        78.828003
2014-09-25 12:15:00   90.530578        40.0      0.288231        79.744736
2014-09-25 12:15:00   90.530578        50.0      0.381517        80.661453
...                         ...         ...           ...              ...
2014-09-25 12:15:00   90.530578       760.0      0.885931        81.360359
2014-09-25 12:15:00   90.530578       770.0      0.892351        81.354958
2014-09-25 12:15:00   90.530578       780.0      0.898783        81.349564
2014-09-25 12:15:00   90.530578       790.0      0.905216        81.344177
2014-09-25 12:15:00   90.530578       800.0      0.911649   

...

[80 rows x 4 columns]
93.28495704044897
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-30 09:15:00   93.284957        10.0      0.118678        78.843544
2014-09-30 09:15:00   93.284957        20.0      0.165894        79.760269
2014-09-30 09:15:00   93.284957        30.0      0.219408        80.677002
2014-09-30 09:15:00   93.284957        40.0      0.289326        81.593719
2014-09-30 09:15:00   93.284957        50.0      0.382612        82.510437
...                         ...         ...           ...              ...
2014-09-30 09:15:00   93.284957       760.0      0.893001        83.770874
2014-09-30 09:15:00   93.284957       770.0      0.899421        83.765472
2014-09-30 09:15:00   93.284957       780.0      0.905854        83.760078
2014-09-30 09:15:00   93.284957       790.0      0.912287        83.754684
2014-09-30 09:15:00   93.284957       800.0      0.918720   

[80 rows x 4 columns]
63.19649721880754
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-30 12:25:00   63.196497        10.0      0.118678        57.843597
2014-09-30 12:25:00   63.196497        20.0      0.176925        57.838192
2014-09-30 12:25:00   63.196497        30.0      0.311755        57.832802
2014-09-30 12:25:00   63.196497        40.0      0.397455        57.827396
2014-09-30 12:25:00   63.196497        50.0      0.492664        57.821995
...                         ...         ...           ...              ...
2014-09-30 12:25:00   63.196497       760.0      1.260515        57.438663
2014-09-30 12:25:00   63.196497       770.0      1.259169        57.433270
2014-09-30 12:25:00   63.196497       780.0      1.257825        57.427860
2014-09-30 12:25:00   63.196497       790.0      1.256481        57.422474
2014-09-30 12:25:00   63.196497       800.0      1.255138   

[80 rows x 4 columns]
106.63258659412463
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-30 15:55:00  106.632587        10.0      0.118678        87.499329
2014-09-30 15:55:00  106.632587        20.0      0.202830        88.400925
2014-09-30 15:55:00  106.632587        30.0      0.250265        89.302521
2014-09-30 15:55:00  106.632587        40.0      0.309207        90.204132
2014-09-30 15:55:00  106.632587        50.0      0.387919        91.105736
...                         ...         ...           ...              ...
2014-09-30 15:55:00  106.632587       760.0      0.927266        94.013947
2014-09-30 15:55:00  106.632587       770.0      0.933686        93.993423
2014-09-30 15:55:00  106.632587       780.0      0.940119        93.972931
2014-09-30 15:55:00  106.632587       790.0      0.946551        93.952408
2014-09-30 15:55:00  106.632587       800.0      0.952984  

                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-30 19:05:00  139.605848        10.0      0.674172       103.974564
2014-09-30 19:05:00  139.605848        20.0      0.699226       104.876167
2014-09-30 19:05:00  139.605848        30.0      0.705864       105.777756
2014-09-30 19:05:00  139.605848        40.0      0.712503       106.679375
2014-09-30 19:05:00  139.605848        50.0      0.719141       107.580963
...                         ...         ...           ...              ...
2014-09-30 19:05:00  139.605848       760.0      1.008534       117.211502
2014-09-30 19:05:00  139.605848       770.0      0.995810       117.190979
2014-09-30 19:05:00  139.605848       780.0      0.997518       117.170471
2014-09-30 19:05:00  139.605848       790.0      1.000526       117.149956
2014-09-30 19:05:00  139.605848       800.0      1.003535       117.129425

[80 rows x 4 columns]
14

[80 rows x 4 columns]
102.5445557825764
                     send rates  block size  latency_pred  throughput_pred
datetime                                                                  
2014-09-30 22:45:00  102.544556        10.0      0.118678        85.059402
2014-09-30 22:45:00  102.544556        20.0      0.183444        85.976128
2014-09-30 22:45:00  102.544556        30.0      0.223725        86.892853
2014-09-30 22:45:00  102.544556        40.0      0.293008        87.809578
2014-09-30 22:45:00  102.544556        50.0      0.386294        88.726295
...                         ...         ...           ...              ...
2014-09-30 22:45:00  102.544556       760.0      0.916771        91.137909
2014-09-30 22:45:00  102.544556       770.0      0.923191        91.117393
2014-09-30 22:45:00  102.544556       780.0      0.929624        91.096870
2014-09-30 22:45:00  102.544556       790.0      0.936057        91.076370
2014-09-30 22:45:00  102.544556       800.0      0.942490   

In [313]:
la_max_min = y.groupby('send rates', sort=False)['latency_pred'].agg(['max', 'min'])
la_max_min

,max,min
send rates,,
63.671638,1.318324,0.118678
61.886153,1.320545,0.118678
63.354798,1.323959,0.118678
58.207375,1.299180,0.118678
54.164232,1.276854,0.118678
...,...,...
48.531916,1.257972,0.118678
49.007470,1.258557,0.118678
46.962178,1.249579,0.118678


In [314]:
la_col=['latency_max', 'latency_min']
la_max_min.columns = la_col
la_max_min

,latency_max,latency_min
send rates,,
63.671638,1.318324,0.118678
61.886153,1.320545,0.118678
63.354798,1.323959,0.118678
58.207375,1.299180,0.118678
54.164232,1.276854,0.118678
...,...,...
48.531916,1.257972,0.118678
49.007470,1.258557,0.118678
46.962178,1.249579,0.118678


In [315]:
th_max_min = y.groupby('send rates', sort=False)['throughput_pred'].agg(['max', 'min'])
th_max_min

,max,min
send rates,,
63.671638,58.259411,57.832890
61.886153,56.696831,56.270306
63.354798,57.982128,57.555607
58.207375,53.477314,53.050781
54.164232,49.938915,49.512383
...,...,...
48.531916,45.009739,44.583210
49.007470,45.425922,44.999390
46.962178,43.635967,43.209438


In [316]:
th_col=['throughput_max', 'throughput_min']
th_max_min.columns = th_col
th_max_min

,throughput_max,throughput_min
send rates,,
63.671638,58.259411,57.832890
61.886153,56.696831,56.270306
63.354798,57.982128,57.555607
58.207375,53.477314,53.050781
54.164232,49.938915,49.512383
...,...,...
48.531916,45.009739,44.583210
49.007470,45.425922,44.999390
46.962178,43.635967,43.209438


In [317]:
# Restore the grouping index to the column
la_max_min.reset_index(inplace=True)
la_max_min

,send rates,latency_max,latency_min
0,63.671638,1.318324,0.118678
1,61.886153,1.320545,0.118678
2,63.354798,1.323959,0.118678
3,58.207375,1.299180,0.118678
4,54.164232,1.276854,0.118678
...,...,...,...
1723,48.531916,1.257972,0.118678
1724,49.007470,1.258557,0.118678
1725,46.962178,1.249579,0.118678
1726,49.080891,1.258581,0.118678


In [318]:
th_max_min.reset_index(inplace=True)
th_max_min

,send rates,throughput_max,throughput_min
0,63.671638,58.259411,57.832890
1,61.886153,56.696831,56.270306
2,63.354798,57.982128,57.555607
3,58.207375,53.477314,53.050781
4,54.164232,49.938915,49.512383
...,...,...,...
1723,48.531916,45.009739,44.583210
1724,49.007470,45.425922,44.999390
1725,46.962178,43.635967,43.209438
1726,49.080891,45.490177,45.063656


In [319]:
la_max = la_max_min['latency_max'].values
la_max = list(la_max)
la_max

[1.318324,
 1.3205453,
 1.3239594,
 1.2991799,
 1.2768537,
 1.2617896,
 1.2417576,
 1.2676073,
 1.2671175,
 1.2472699,
 1.2251723,
 1.231472,
 1.2207917,
 1.2176977,
 1.2104027,
 1.2005547,
 1.1948451,
 1.190788,
 1.1773612,
 1.181351,
 1.1836326,
 1.1840632,
 1.1855671,
 1.1898391,
 1.1882757,
 1.1917697,
 1.1909939,
 1.1945527,
 1.1956942,
 1.199762,
 1.1999142,
 1.2013588,
 1.1976187,
 1.2005504,
 1.2080644,
 1.2046019,
 1.2041787,
 1.19757,
 1.1968015,
 1.2024721,
 1.2012082,
 1.1990502,
 1.2016445,
 1.2019869,
 1.2047595,
 1.2049339,
 1.2036911,
 1.2018956,
 1.2012209,
 1.1989578,
 1.1990956,
 1.200273,
 1.1997911,
 1.1955949,
 1.1942362,
 1.1897045,
 1.1908356,
 1.1968502,
 1.1949856,
 1.1904004,
 1.1898246,
 1.1844791,
 1.1783665,
 1.1838474,
 1.1841755,
 1.1871846,
 1.1927304,
 1.1951785,
 1.2008909,
 1.2014656,
 1.2163815,
 1.2168978,
 1.224032,
 1.2487894,
 1.2584441,
 1.2602745,
 1.2675389,
 1.2911884,
 1.3156062,
 1.257624,
 1.0985794,
 1.1597201,
 1.086478,
 1.1472505,
 1.

In [320]:
la_max_all = []
for i in range(len(la_max)):
    la_max_all.extend([la_max[i]] * 80)
    
la_max_all

[1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.318324,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,
 1.3205453,


In [321]:
la_min = la_max_min['latency_min'].values
la_min = list(la_min)
la_min

[0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.1318392,
 0.14565061,
 0.15354967,
 0.15503903,
 0.16041371,
 0.18492195,
 0.174734,
 0.19664747,
 0.19234651,
 0.21602327,
 0.22409767,
 0.2528625,
 0.25394142,
 0.2645317,
 0.23770894,
 0.25844085,
 0.31425595,
 0.28913647,
 0.2859255,
 0.23736304,
 0.23192771,
 0.27297685,
 0.26338816,
 0.24782988,
 0.26669514,
 0.26929465,
 0.2903332,
 0.29165766,
 0.28222582,
 0.268601,
 0.26348248,
 0.24717808,
 0.24815136,
 0.25647798,
 0.25306875,
 0.22339398,
 0.21378656,
 0.18404296,
 0.19141403,
 0.2322723,
 0.21908599,
 0.18857712,
 0.18482646,
 0.15647921,
 0.13531922,
 0.15429267,
 0.12287371,
 0.12014486,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.11867

In [322]:
la_min_all = []
for i in range(len(la_min)):
    la_min_all.extend([la_min[i]] * 80)
    
la_min_all

[0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118677735,
 0.118

In [323]:
y['latency_min'] = la_min_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min
datetime,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259411,0.118678
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248619,0.118678
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243221,0.118678
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678
...,...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727749,0.118678
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722359,0.118678
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678


In [324]:
y['latency_max'] = la_max_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max
datetime,,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259411,0.118678,1.318324
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248619,0.118678,1.318324
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243221,0.118678,1.318324
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324
...,...,...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727749,0.118678,1.259023
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722359,0.118678,1.259023
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023


In [325]:
th_max = th_max_min['throughput_max'].values
th_max = list(th_max)
th_max

[58.25941,
 56.69683,
 57.98213,
 53.477314,
 49.938915,
 46.447582,
 42.161015,
 47.41267,
 47.05399,
 42.788364,
 38.188305,
 39.64452,
 37.120663,
 36.366497,
 35.107803,
 33.339848,
 31.26696,
 29.79403,
 26.965714,
 25.170464,
 24.143715,
 23.950138,
 23.273293,
 21.351002,
 22.054502,
 20.541323,
 20.838303,
 19.475807,
 19.038599,
 17.481218,
 17.422825,
 16.869587,
 18.301647,
 17.17918,
 14.398872,
 15.63886,
 15.799192,
 18.320402,
 18.614658,
 16.445784,
 16.927282,
 17.753695,
 16.760393,
 16.629644,
 15.579097,
 15.512954,
 15.983944,
 16.664288,
 16.922527,
 17.78899,
 17.736284,
 17.285427,
 17.470043,
 19.076729,
 19.596897,
 21.411694,
 20.902704,
 18.596031,
 19.309978,
 21.098595,
 21.357574,
 23.762932,
 26.513367,
 24.047136,
 28.131071,
 28.48582,
 30.499268,
 31.38798,
 33.461964,
 33.6706,
 36.04572,
 36.17159,
 37.91045,
 42.961353,
 45.112118,
 46.27515,
 47.223347,
 52.21626,
 55.99738,
 61.24615,
 69.061165,
 66.052734,
 69.78093,
 66.6663,
 69.28868,
 78.11

In [326]:
th_max_all = []
for i in range(len(th_max)):
    th_max_all.extend([th_max[i]] * 80)
    
th_max_all

[58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 58.25941,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,
 56.69683,

In [327]:
th_min = th_max_min['throughput_min'].values
th_min = list(th_min)
th_min

[57.83289,
 56.270306,
 57.555607,
 53.05078,
 49.512383,
 46.02105,
 41.734493,
 46.98615,
 46.627476,
 42.361835,
 37.761772,
 39.217987,
 36.694126,
 35.939972,
 34.681267,
 32.91332,
 30.840439,
 29.367502,
 26.53918,
 24.74393,
 23.71718,
 23.52361,
 22.84676,
 20.924473,
 21.627977,
 20.1148,
 20.41178,
 19.049274,
 18.612076,
 17.05469,
 16.9963,
 16.443062,
 17.875122,
 16.752645,
 13.933993,
 15.2015295,
 15.363535,
 17.893871,
 18.188133,
 16.01691,
 16.500751,
 17.327168,
 16.333862,
 16.20271,
 15.141141,
 15.074296,
 15.550229,
 16.237719,
 16.496,
 17.362463,
 17.30975,
 16.858898,
 17.043522,
 18.6502,
 19.17037,
 20.985165,
 20.476173,
 18.169506,
 18.883455,
 20.67207,
 20.931044,
 23.336409,
 26.086842,
 23.620613,
 27.704542,
 28.059288,
 30.072733,
 30.96146,
 33.03544,
 33.24407,
 35.619194,
 35.74506,
 37.48392,
 42.53483,
 44.68559,
 45.84863,
 46.796818,
 51.789722,
 55.57086,
 60.819614,
 67.258194,
 64.95059,
 67.81444,
 65.42122,
 67.43685,
 74.21826,
 76.909

In [328]:
th_min_all = []
for i in range(len(th_min)):
    th_min_all.extend([th_min[i]] * 80)
    
th_min_all

[57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 57.83289,
 56.270306,
 56.270306,
 56.270306,
 56.270306,
 56.270306,
 56.270306,
 56.270306,
 56.270306,
 56.270306,
 56.270306,


In [329]:
y['throughput_min'] = th_min_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min
datetime,,,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259411,0.118678,1.318324,57.83289
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324,57.83289
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248619,0.118678,1.318324,57.83289
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243221,0.118678,1.318324,57.83289
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324,57.83289
...,...,...,...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727749,0.118678,1.259023,45.70615
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722359,0.118678,1.259023,45.70615
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023,45.70615


In [330]:
y['throughput_max'] = th_max_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max
datetime,,,,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259411,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248619,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243221,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324,57.83289,58.259411
...,...,...,...,...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727749,0.118678,1.259023,45.70615,46.132683
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722359,0.118678,1.259023,45.70615,46.132683
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023,45.70615,46.132683


In [331]:
y.to_csv('./latency_throughput_pred_max_min.csv')

### choose different weights

In [478]:
# Read the processed data directly
y = pd.read_csv('./latency_throughput_pred_max_min.csv')
y.set_index('datetime', drop=True, inplace=True)
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max
datetime,,,,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248620,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243220,0.118678,1.318324,57.83289,58.259411
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324,57.83289,58.259411
...,...,...,...,...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727750,0.118678,1.259023,45.70615,46.132683
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722360,0.118678,1.259023,45.70615,46.132683
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023,45.70615,46.132683


In [479]:
alpha = 0.9
beta = 0.1

In [480]:
latency_score = alpha * ((y['latency_max'] - y['latency_pred']) / (y['latency_max'] - y['latency_min']))

In [481]:
throughput_score = beta * ((y['throughput_pred'] - y['throughput_min']) / (y['throughput_max'] - y['throughput_min']))

In [482]:
y_score = latency_score.values + throughput_score.values
y_score

array([0.9999998 , 0.95621437, 0.85560207, ..., 0.04304326, 0.04283859,
       0.04263338])

In [483]:
y['score'] = y_score

In [484]:
y.to_csv('./latency_throughput_score_alpha' + str(alpha) + '_beta' + str(beta) + '.csv')

In [485]:
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score
datetime,,,,,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.83289,58.259411,1.000000
2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324,57.83289,58.259411,0.956214
2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248620,0.118678,1.318324,57.83289,58.259411,0.855602
2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243220,0.118678,1.318324,57.83289,58.259411,0.791134
2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324,57.83289,58.259411,0.718441
...,...,...,...,...,...,...,...,...,...
2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727750,0.118678,1.259023,45.70615,46.132683,0.043453
2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722360,0.118678,1.259023,45.70615,46.132683,0.043252
2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023,45.70615,46.132683,0.043043


In [486]:
y_filter = y[y['send rates'] > 100]
y_filter

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score
datetime,,,,,,,,,
2014-09-25 07:35:00,100.469344,10.0,0.118678,83.666350,0.118678,0.977915,83.666351,90.420105,0.900000
2014-09-25 07:35:00,100.469344,20.0,0.174778,84.583070,0.118678,0.977915,83.666351,90.420105,0.854812
2014-09-25 07:35:00,100.469344,30.0,0.215704,85.499790,0.118678,0.977915,83.666351,90.420105,0.825517
2014-09-25 07:35:00,100.469344,40.0,0.292183,86.416504,0.118678,0.977915,83.666351,90.420105,0.758985
2014-09-25 07:35:00,100.469344,50.0,0.385469,87.333240,0.118678,0.977915,83.666351,90.420105,0.674847
...,...,...,...,...,...,...,...,...,...
2014-09-30 22:50:00,104.514599,760.0,0.921829,92.523895,0.118678,0.976228,86.381882,93.878075,0.139027
2014-09-30 22:50:00,104.514599,770.0,0.928249,92.503370,0.118678,0.976228,86.381882,93.878075,0.132015
2014-09-30 22:50:00,104.514599,780.0,0.934682,92.482860,0.118678,0.976228,86.381882,93.878075,0.124990


In [487]:
# for name, group in y_filter.groupby('send rates', sort=False):
#     print(name)
#     print(group)

In [488]:
score_max = y_filter.groupby('send rates', sort=False)['score'].agg(['max'])
score_max

,max
send rates,
100.469344,0.9
102.594089,0.9
102.325191,0.9
102.499313,0.9
101.160802,0.9
...,...
105.341173,0.9
104.504316,0.9
100.911508,0.9


In [489]:
score_col=['score_max']
score_max.columns = score_col
score_max

,score_max
send rates,
100.469344,0.9
102.594089,0.9
102.325191,0.9
102.499313,0.9
101.160802,0.9
...,...
105.341173,0.9
104.504316,0.9
100.911508,0.9


In [490]:
score_max.reset_index(inplace=True)
score_max

,send rates,score_max
0,100.469344,0.9
1,102.594089,0.9
2,102.325191,0.9
3,102.499313,0.9
4,101.160802,0.9
...,...,...
584,105.341173,0.9
585,104.504316,0.9
586,100.911508,0.9
587,102.544556,0.9


In [491]:
score_max = score_max['score_max'].values
score_max = list(score_max)
score_max

[0.8999999808238718,
 0.8999999813221011,
 0.89999999822285,
 0.8999999988806164,
 0.900000023219303,
 0.9000000053738033,
 0.8999999926434695,
 0.899999955196275,
 0.900000023336804,
 0.9000000485289218,
 0.8999999999707006,
 0.9000000202720274,
 0.8999999960232166,
 0.899999965516701,
 0.8999999979004079,
 0.9000000007392823,
 0.8999999840004236,
 0.9000000040459679,
 0.9000000302153708,
 0.8999999870989324,
 0.9000000202467074,
 0.9000000403916524,
 0.9000000294747715,
 0.9000000146105399,
 0.9000000046848179,
 0.9000000145175031,
 0.8999999741103386,
 0.9000000037492817,
 0.8999999966101787,
 0.8999999869876676,
 0.9000000072445632,
 0.8999999859693147,
 0.8999999767185991,
 0.8999999674374901,
 0.9000000010587266,
 0.8999999680475474,
 0.8999999676184273,
 0.9000000300229511,
 0.8999999737114167,
 0.899999995728802,
 0.899999966955102,
 0.9000000317726287,
 0.8999999600801266,
 0.8999999914504881,
 0.8999999913009531,
 0.8999999887011152,
 0.9000000335032109,
 0.9000000070705979,


In [492]:
score_max_all = []
for i in range(len(score_max)):
    score_max_all.extend([score_max[i]] * 80)
    
score_max_all

[0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808238718,
 0.8999999808

In [493]:
y_filter['score_max'] = score_max_all
y_filter

<ipython-input-493-d3f22a348827>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_filter['score_max'] = score_max_all


,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
datetime,,,,,,,,,,
2014-09-25 07:35:00,100.469344,10.0,0.118678,83.666350,0.118678,0.977915,83.666351,90.420105,0.900000,0.9
2014-09-25 07:35:00,100.469344,20.0,0.174778,84.583070,0.118678,0.977915,83.666351,90.420105,0.854812,0.9
2014-09-25 07:35:00,100.469344,30.0,0.215704,85.499790,0.118678,0.977915,83.666351,90.420105,0.825517,0.9
2014-09-25 07:35:00,100.469344,40.0,0.292183,86.416504,0.118678,0.977915,83.666351,90.420105,0.758985,0.9
2014-09-25 07:35:00,100.469344,50.0,0.385469,87.333240,0.118678,0.977915,83.666351,90.420105,0.674847,0.9
...,...,...,...,...,...,...,...,...,...,...
2014-09-30 22:50:00,104.514599,760.0,0.921829,92.523895,0.118678,0.976228,86.381882,93.878075,0.139027,0.9
2014-09-30 22:50:00,104.514599,770.0,0.928249,92.503370,0.118678,0.976228,86.381882,93.878075,0.132015,0.9
2014-09-30 22:50:00,104.514599,780.0,0.934682,92.482860,0.118678,0.976228,86.381882,93.878075,0.124990,0.9


In [494]:
result = y_filter[y_filter['score'] == y_filter['score_max']]
result

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
datetime,,,,,,,,,,
2014-09-25 07:35:00,100.469344,10.0,0.118678,83.666350,0.118678,0.977915,83.666351,90.420105,0.9,0.9
2014-09-25 07:45:00,102.594089,10.0,0.118678,85.092650,0.118678,0.975309,85.092651,92.279594,0.9,0.9
2014-09-25 07:50:00,102.325191,10.0,0.118678,84.912155,0.118678,0.975638,84.912155,92.044273,0.9,0.9
2014-09-25 07:55:00,102.499313,10.0,0.118678,85.029045,0.118678,0.975425,85.029045,92.196655,0.9,0.9
2014-09-25 08:05:00,101.160802,10.0,0.118678,84.130510,0.118678,0.977067,84.130508,91.025238,0.9,0.9
...,...,...,...,...,...,...,...,...,...,...
2014-09-30 22:30:00,105.341173,10.0,0.118678,86.854065,0.118678,0.979822,86.854065,94.459587,0.9,0.9
2014-09-30 22:35:00,104.504316,10.0,0.118678,86.374980,0.118678,0.976183,86.374977,93.870834,0.9,0.9
2014-09-30 22:40:00,100.911508,10.0,0.118678,83.963165,0.118678,0.977373,83.963165,90.807068,0.9,0.9


In [495]:
result.to_csv('./best_block_size_alpha' + str(alpha) + '_beta' + str(beta) + '.csv')

### Select the block size of 100, 200, 400 and the block size with the largest score for comparison experiments

In [1]:
import pandas as pd

In [108]:
alpha = 0.9
beta = 0.1

In [109]:
y = pd.read_csv('./result/latency_throughput_score_alpha' + str(alpha) + '_beta' + str(beta) + '.csv')

In [110]:
y

,datetime,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score
0,2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.83289,58.259411,1.000000
1,2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324,57.83289,58.259411,0.956214
2,2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248620,0.118678,1.318324,57.83289,58.259411,0.855602
3,2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243220,0.118678,1.318324,57.83289,58.259411,0.791134
4,2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324,57.83289,58.259411,0.718441
...,...,...,...,...,...,...,...,...,...,...
138235,2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727750,0.118678,1.259023,45.70615,46.132683,0.043453
138236,2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722360,0.118678,1.259023,45.70615,46.132683,0.043252
138237,2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023,45.70615,46.132683,0.043043
138238,2014-10-01 00:00:00,49.815040,790.0,1.206348,45.711550,0.118678,1.259023,45.70615,46.132683,0.042839


In [111]:
# for name, group in y_filter.groupby('send rates', sort=False):
#     print(name)
#     print(group)

In [112]:
score_max = y.groupby('send rates', sort=False)['score'].agg(['max'])
score_max

,max
send rates,
63.671638,1.0
61.886153,1.0
63.354798,1.0
58.207375,1.0
54.164232,1.0
...,...
48.531916,1.0
49.007470,1.0
46.962178,1.0


In [113]:
score_col=['score_max']
score_max.columns = score_col
score_max

,score_max
send rates,
63.671638,1.0
61.886153,1.0
63.354798,1.0
58.207375,1.0
54.164232,1.0
...,...
48.531916,1.0
49.007470,1.0
46.962178,1.0


In [114]:
score_max.reset_index(inplace=True)
score_max

,send rates,score_max
0,63.671638,1.0
1,61.886153,1.0
2,63.354798,1.0
3,58.207375,1.0
4,54.164232,1.0
...,...,...
1723,48.531916,1.0
1724,49.007470,1.0
1725,46.962178,1.0
1726,49.080891,1.0


In [115]:
score_max = score_max['score_max'].values
score_max = list(score_max)
score_max

[0.9999997990481176,
 0.9999998245210148,
 1.000000435555326,
 1.0000000013381072,
 0.9999999410136384,
 0.9999999428486326,
 1.0000001041539173,
 0.9999999684892846,
 1.0000001384945505,
 0.9999999040434504,
 1.000000023448899,
 1.0000000457703715,
 1.000000073132265,
 0.9999999909391256,
 0.99999991945102,
 1.0000001023304452,
 0.999999966503294,
 1.000000179434584,
 1.0000001177307356,
 1.0000001029705157,
 1.0000000233478377,
 0.9999999760885494,
 1.0000001094805837,
 1.0000000611651636,
 0.9999998825742404,
 1.0000000641151503,
 1.0000000938448372,
 0.999999952079659,
 0.9999999958405884,
 0.99999993317091,
 1.000000029811511,
 1.0000000147256771,
 0.9999999573595488,
 0.9999999680117322,
 0.9999999220835428,
 1.000000056805115,
 0.9999999090180018,
 0.9999999663335578,
 0.9999999145419526,
 1.0000000853895252,
 0.9999999188568348,
 1.0000001132434146,
 0.9999999682692906,
 0.9999999071177662,
 1.0000000582491522,
 1.0000000515967165,
 1.0000000115178844,
 1.0000001017730729,
 0.9

In [116]:
score_max_all = []
for i in range(len(score_max)):
    score_max_all.extend([score_max[i]] * 80)
    
score_max_all

[0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990481176,
 0.9999997990

In [117]:
y['score_max'] = score_max_all
y

,datetime,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
0,2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.83289,58.259411,1.000000,1.0
1,2014-09-25 00:05:00,63.671638,20.0,0.175356,58.254017,0.118678,1.318324,57.83289,58.259411,0.956214,1.0
2,2014-09-25 00:05:00,63.671638,30.0,0.307779,58.248620,0.118678,1.318324,57.83289,58.259411,0.855602,1.0
3,2014-09-25 00:05:00,63.671638,40.0,0.392023,58.243220,0.118678,1.318324,57.83289,58.259411,0.791134,1.0
4,2014-09-25 00:05:00,63.671638,50.0,0.487231,58.237820,0.118678,1.318324,57.83289,58.259411,0.718441,1.0
...,...,...,...,...,...,...,...,...,...,...,...
138235,2014-10-01 00:00:00,49.815040,760.0,1.210382,45.727750,0.118678,1.259023,45.70615,46.132683,0.043453,1.0
138236,2014-10-01 00:00:00,49.815040,770.0,1.209036,45.722360,0.118678,1.259023,45.70615,46.132683,0.043252,1.0
138237,2014-10-01 00:00:00,49.815040,780.0,1.207692,45.716946,0.118678,1.259023,45.70615,46.132683,0.043043,1.0
138238,2014-10-01 00:00:00,49.815040,790.0,1.206348,45.711550,0.118678,1.259023,45.70615,46.132683,0.042839,1.0


In [118]:
result = y[(y['score'] == y['score_max']) | (y['block size'] == 100.0) | (y['block size'] == 200.0) | (y['block size'] == 400.0)]
result

,datetime,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
0,2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.832890,58.259411,1.000000,1.0
9,2014-09-25 00:05:00,63.671638,100.0,1.008658,58.210820,0.118678,1.318324,57.832890,58.259411,0.320925,1.0
19,2014-09-25 00:05:00,63.671638,200.0,1.216315,58.156834,0.118678,1.318324,57.832890,58.259411,0.152480,1.0
39,2014-09-25 00:05:00,63.671638,400.0,1.207118,58.048855,0.118678,1.318324,57.832890,58.259411,0.134063,1.0
80,2014-09-25 00:10:00,61.886153,10.0,0.118678,56.696830,0.118678,1.320545,56.270306,56.696831,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
138119,2014-09-30 23:55:00,49.080891,400.0,1.197305,45.279613,0.118678,1.258581,45.063656,45.490177,0.099012,1.0
138160,2014-10-01 00:00:00,49.815040,10.0,0.118678,46.132683,0.118678,1.259023,45.706150,46.132683,1.000000,1.0
138169,2014-10-01 00:00:00,49.815040,100.0,1.070648,46.084084,0.118678,1.259023,45.706150,46.132683,0.237278,1.0
138179,2014-10-01 00:00:00,49.815040,200.0,1.241392,46.030090,0.118678,1.259023,45.706150,46.132683,0.089862,1.0


In [119]:
result.to_csv('./result/compare_block_size_alpha' + str(alpha) + '_beta' + str(beta) + '.csv')

# Add time index to transaction sending rate

In [120]:
send_rates_pred = pd.read_csv('../result/y_true_pred.csv')

In [121]:
send_rates_pred

,Unnamed: 0,y_true,y_pred
0,0,17937.000832,19101.491395
1,1,19263.999929,18565.845787
2,2,16234.000265,19006.439372
3,3,14814.000125,17462.212608
4,4,13504.000112,16249.269695
...,...,...,...
1723,1723,15835.999871,14559.574796
1724,1724,13712.000203,14702.240965
1725,1725,14814.999909,14088.653506
1726,1726,14928.000307,14724.267431


In [122]:
send_rates_pred['y_true'] = send_rates_pred['y_true'] / 300
send_rates_pred['y_pred'] = send_rates_pred['y_pred'] / 300

send_rates_pred

,Unnamed: 0,y_true,y_pred
0,0,59.790003,63.671638
1,1,64.213333,61.886153
2,2,54.113334,63.354798
3,3,49.380000,58.207375
4,4,45.013334,54.164232
...,...,...,...
1723,1723,52.786666,48.531916
1724,1724,45.706667,49.007470
1725,1725,49.383333,46.962178
1726,1726,49.760001,49.080891


In [123]:
rng = pd.date_range('2014/09/25 00:05',periods=1728,freq ='5T')
send_rates_pred.drop("Unnamed: 0", axis=1, inplace=True)
send_rates_pred.index = rng

send_rates_pred

,y_true,y_pred
2014-09-25 00:05:00,59.790003,63.671638
2014-09-25 00:10:00,64.213333,61.886153
2014-09-25 00:15:00,54.113334,63.354798
2014-09-25 00:20:00,49.380000,58.207375
2014-09-25 00:25:00,45.013334,54.164232
...,...,...
2014-09-30 23:40:00,52.786666,48.531916
2014-09-30 23:45:00,45.706667,49.007470
2014-09-30 23:50:00,49.383333,46.962178
2014-09-30 23:55:00,49.760001,49.080891


In [124]:
send_rates_pred.to_csv('../result/send_rates_pred_truth.csv')

# Optimal block for all time slices

In [125]:
import pandas as pd

In [165]:
alpha = 0.9
beta = 0.1

In [166]:
com = pd.read_csv('./result/compare_block_size/compare_block_size_alpha' + str(alpha) + '_beta' + str(beta) + '.csv')
com

,Unnamed: 0,datetime,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
0,0,2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.832890,58.259411,1.000000,1.0
1,9,2014-09-25 00:05:00,63.671638,100.0,1.008658,58.210820,0.118678,1.318324,57.832890,58.259411,0.320925,1.0
2,19,2014-09-25 00:05:00,63.671638,200.0,1.216315,58.156834,0.118678,1.318324,57.832890,58.259411,0.152480,1.0
3,39,2014-09-25 00:05:00,63.671638,400.0,1.207118,58.048855,0.118678,1.318324,57.832890,58.259411,0.134063,1.0
4,80,2014-09-25 00:10:00,61.886153,10.0,0.118678,56.696830,0.118678,1.320545,56.270306,56.696831,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6907,138119,2014-09-30 23:55:00,49.080891,400.0,1.197305,45.279613,0.118678,1.258581,45.063656,45.490177,0.099012,1.0
6908,138160,2014-10-01 00:00:00,49.815040,10.0,0.118678,46.132683,0.118678,1.259023,45.706150,46.132683,1.000000,1.0
6909,138169,2014-10-01 00:00:00,49.815040,100.0,1.070648,46.084084,0.118678,1.259023,45.706150,46.132683,0.237278,1.0
6910,138179,2014-10-01 00:00:00,49.815040,200.0,1.241392,46.030090,0.118678,1.259023,45.706150,46.132683,0.089862,1.0


In [167]:
result = com[com['score'] == com['score_max']]

result

,Unnamed: 0,datetime,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
0,0,2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.832890,58.259411,1.0,1.0
4,80,2014-09-25 00:10:00,61.886153,10.0,0.118678,56.696830,0.118678,1.320545,56.270306,56.696831,1.0,1.0
8,160,2014-09-25 00:15:00,63.354798,10.0,0.118678,57.982130,0.118678,1.323959,57.555607,57.982128,1.0,1.0
12,240,2014-09-25 00:20:00,58.207375,10.0,0.118678,53.477314,0.118678,1.299180,53.050781,53.477314,1.0,1.0
16,320,2014-09-25 00:25:00,54.164232,10.0,0.118678,49.938915,0.118678,1.276854,49.512383,49.938915,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6892,137840,2014-09-30 23:40:00,48.531916,10.0,0.118678,45.009740,0.118678,1.257972,44.583210,45.009739,1.0,1.0
6896,137920,2014-09-30 23:45:00,49.007470,10.0,0.118678,45.425922,0.118678,1.258557,44.999390,45.425922,1.0,1.0
6900,138000,2014-09-30 23:50:00,46.962178,10.0,0.118678,43.635967,0.118678,1.249579,43.209438,43.635967,1.0,1.0
6904,138080,2014-09-30 23:55:00,49.080891,10.0,0.118678,45.490177,0.118678,1.258581,45.063656,45.490177,1.0,1.0


In [168]:
result.drop('Unnamed: 0', axis=1, inplace=True)
result.set_index('datetime', drop=True, inplace=True)

result

D:\Anaconda3\envs\d2l-zh\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
datetime,,,,,,,,,,
2014-09-25 00:05:00,63.671638,10.0,0.118678,58.259410,0.118678,1.318324,57.832890,58.259411,1.0,1.0
2014-09-25 00:10:00,61.886153,10.0,0.118678,56.696830,0.118678,1.320545,56.270306,56.696831,1.0,1.0
2014-09-25 00:15:00,63.354798,10.0,0.118678,57.982130,0.118678,1.323959,57.555607,57.982128,1.0,1.0
2014-09-25 00:20:00,58.207375,10.0,0.118678,53.477314,0.118678,1.299180,53.050781,53.477314,1.0,1.0
2014-09-25 00:25:00,54.164232,10.0,0.118678,49.938915,0.118678,1.276854,49.512383,49.938915,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2014-09-30 23:40:00,48.531916,10.0,0.118678,45.009740,0.118678,1.257972,44.583210,45.009739,1.0,1.0
2014-09-30 23:45:00,49.007470,10.0,0.118678,45.425922,0.118678,1.258557,44.999390,45.425922,1.0,1.0
2014-09-30 23:50:00,46.962178,10.0,0.118678,43.635967,0.118678,1.249579,43.209438,43.635967,1.0,1.0


In [169]:
result.to_csv('./result/all_best_block_size/all_best_block_size_alpha' + str(alpha) + '_beta' + str(beta) + '.csv')